In [16]:
!pip install QuantTraderLib==1.0.23

  Using cached QuantTraderLib-1.0.22-py3-none-any.whl.metadata (1.0 kB)
Using cached QuantTraderLib-1.0.22-py3-none-any.whl (64 kB)
  Attempting uninstall: QuantTraderLib
    Found existing installation: QuantTraderLib 1.0.17
    Uninstalling QuantTraderLib-1.0.17:
      Successfully uninstalled QuantTraderLib-1.0.17


## Tạo dữ liệu

In [25]:
from QuantTraderLib.backtest.event_base import use_changes
from QuantTraderLib.backtest.vectorized import use_position, use_signal_ma, use_trailing

In [19]:
from QuantTraderLib.data.get_data import *

Lấy dữ liệu bitcoin

In [20]:
data = get_crypto('BTC', dt.datetime(2023,1,1))

[*********************100%%**********************]  1 of 1 completed


In [21]:
data

,Date,Open,High,Low,Close
0,2024-07-31,5.90,5.930,5.760,5.79
1,2024-08-01,5.73,5.760,5.520,5.62
2,2024-08-02,5.75,5.810,5.530,5.56
3,2024-08-05,4.41,4.930,4.400,4.75
4,2024-08-06,4.90,5.070,4.830,5.05
5,2024-08-07,5.09,5.110,4.850,4.86
6,2024-08-08,5.13,5.315,5.040,5.27
7,2024-08-09,5.37,5.425,5.288,5.38
8,2024-08-12,5.29,5.385,5.130,5.24
9,2024-08-13,5.23,5.465,5.220,5.41


In [22]:
selected_columns = data[['Open', 'High', 'Low', 'Close']] # Một số hàm backtest chỉ nhận các cột dữ liệu này

Tạo series vị thế ngẫu nhiên để sử dụng chiến thuật phía sau

In [23]:
import pandas as pd
import numpy as np

# Generate random integers (-1, 0, 1)
random_pos = np.random.randint(-1, 2, size=len(data.Close))

# Convert to pandas Series
random_pos = pd.Series(random_pos)

## Backtesting

### Backtest with use_position

Backtest dữ liệu sử dụng chiến thuật vị thế

In [26]:
stats, bt = use_position(selected_columns, random_pos) # Input là data và dãy vị thế của giá Close

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.


In [27]:
stats

Start                                     0.0
End                                      43.0
Duration                                 43.0
Exposure Time [%]                   56.818182
Equity Final [$]                   9933.72882
Equity Peak [$]                   10178.44942
Return [%]                          -0.662712
Buy & Hold Return [%]               -5.354058
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -13.978284
Avg. Drawdown [%]                   -8.191293
Max. Drawdown Duration                   33.0
Avg. Drawdown Duration                   17.5
# Trades                                  8.0
Win Rate [%]                             50.0
Best Trade [%]                       5.451127
Worst Trade [%]                     -3.525048
Avg. Trade [%]                    

In [28]:
bt.plot()

found multiple competing values for 'toolbar.active_drag' property; using the latest value
found multiple competing values for 'toolbar.active_scroll' property; using the latest value


GridPlot(id='p1241', ...)

### Backtest with use_changes

Sử dụng giá thay đổi được dự đoán để backtest

In [29]:
changes = data['Close'] - data['Close'].shift(1) 

In [30]:
stats, bt = use_changes(data, changes)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.


In [31]:
stats

Start                                     0.0
End                                      43.0
Duration                                 43.0
Exposure Time [%]                   72.727273
Equity Final [$]                 10529.609519
Equity Peak [$]                  10904.210055
Return [%]                           5.296095
Buy & Hold Return [%]               -5.354058
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -5.852744
Avg. Drawdown [%]                   -3.418818
Max. Drawdown Duration                   16.0
Avg. Drawdown Duration               5.142857
# Trades                                 12.0
Win Rate [%]                        58.333333
Best Trade [%]                       5.451127
Worst Trade [%]                     -3.436431
Avg. Trade [%]                    

In [32]:
bt.plot()

found multiple competing values for 'toolbar.active_drag' property; using the latest value
found multiple competing values for 'toolbar.active_scroll' property; using the latest value


GridPlot(id='p1504', ...)

### Backtest with use_changes

Sử dụng 2 chu kỳ moving average khác nhau để tạo signal và tính backtest

In [33]:
stats, bt = use_signal_ma(data, 10, 20)

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.


In [34]:
stats

Start                                     0.0
End                                      43.0
Duration                                 43.0
Exposure Time [%]                   15.909091
Equity Final [$]                  9999.509999
Equity Peak [$]                  10000.710001
Return [%]                            -0.0049
Buy & Hold Return [%]               -5.354058
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.011999
Avg. Drawdown [%]                    -0.00725
Max. Drawdown Duration                    2.0
Avg. Drawdown Duration                    2.0
# Trades                                  7.0
Win Rate [%]                        14.285714
Best Trade [%]                       0.178249
Worst Trade [%]                     -3.436431
Avg. Trade [%]                    

Thực hiện backtest chiến lược giao dịch sử dụng trailing stop-loss.

In [35]:
stats, bt = use_trailing(data=data, atr_periods=40, trailing_sl=3, rolling=10)

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.


In [36]:
stats

Start                                     0.0
End                                      43.0
Duration                                 43.0
Exposure Time [%]                   63.636364
Equity Final [$]                 10339.452478
Equity Peak [$]                  10823.450584
Return [%]                           3.394525
Buy & Hold Return [%]               -5.354058
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -13.137101
Avg. Drawdown [%]                   -7.756427
Max. Drawdown Duration                   26.0
Avg. Drawdown Duration                   14.0
# Trades                                  2.0
Win Rate [%]                             50.0
Best Trade [%]                       9.980425
Worst Trade [%]                     -5.986586
Avg. Trade [%]                    